# Part 2: 모델 임포트 및 Sequential 모델 학습

이 노트북은 Part 1에서 이어집니다.

## 5. 필수 라이브러리 임포트 및 설정

In [1]:
# 필수 라이브러리 임포트
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import os
import gc
import pickle
import json
from datetime import datetime

# 학습 파라미터 정의 (Part 1에서 정의했지만 여기서도 명시)
EPOCHS = 200
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.2
INITIAL_LR = 1e-3
FINAL_LR = INITIAL_LR * 0.1
DECAY_RATE = (FINAL_LR / INITIAL_LR) ** (1 / EPOCHS)

print(f"✓ Libraries imported")
print(f"✓ Training configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Initial LR: {INITIAL_LR}")
print(f"  Final LR: {FINAL_LR}")

2025-11-03 12:27:47.043810: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-03 12:27:47.103164: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-03 12:27:47.103222: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-03 12:27:47.105973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-03 12:27:47.127228: I tensorflow/core/platform/cpu_feature_guar

✓ Libraries imported
✓ Training configuration:
  Epochs: 200
  Batch size: 128
  Initial LR: 0.001
  Final LR: 0.0001


## 6. 유틸리티 함수 로드

In [2]:
# 유틸리티 함수 임포트
from utils import (
    get_callbacks,
    clear_memory,
    save_history,
    save_results
)

# 디렉토리 생성
os.makedirs('weights', exist_ok=True)
os.makedirs('history', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('logs', exist_ok=True)

print("✓ Utility functions loaded")
print("✓ Directories created")

✓ Utility functions loaded
✓ Directories created


## 7. 데이터 로드

In [3]:
# Train data
print("Loading augmented data...")
x_train_augmented = np.load('data/x_train_augmented.npy')
y_train_augmented = np.load('data/y_train_augmented.npy')
x_train_clean = np.load('data/x_train_clean.npy')

# Noise info (original_index 매칭용)
train_noise_info = pd.read_csv('data/train_noise_info.csv')
test_noise_info = pd.read_csv('data/test_noise_info.csv')

# Test data
x_test_augmented = np.load('data/x_test_augmented.npy')
y_test_augmented = np.load('data/y_test_augmented.npy')
x_test_clean = np.load('data/x_test_clean.npy')

print(f"✓ Train shape: {x_train_augmented.shape}")
print(f"✓ Test shape: {x_test_augmented.shape}")

# BAM 모델용 flattened 데이터
x_train_flat = x_train_augmented.reshape(x_train_augmented.shape[0], -1)

# ⚠️ 수정: original_index를 사용하여 올바른 clean target 생성
print("Creating correct clean targets using original_index...")
train_original_indices = train_noise_info['original_index'].values
x_train_clean_matched = x_train_clean[train_original_indices]
x_train_clean_flat = x_train_clean_matched.reshape(-1, 32*32*3)

x_test_flat = x_test_augmented.reshape(x_test_augmented.shape[0], -1)
test_original_indices = test_noise_info['original_index'].values
x_test_clean_matched = x_test_clean[test_original_indices]
x_test_clean_flat = x_test_clean_matched.reshape(-1, 32*32*3)

print(f"✓ Train flat shape: {x_train_flat.shape}")
print(f"✓ Train clean flat shape: {x_train_clean_flat.shape}")
print(f"✓ Test flat shape: {x_test_flat.shape}")
print(f"✓ Test clean flat shape: {x_test_clean_flat.shape}")

# 데이터 검증 (NaN 체크)
print("\n[Data Validation]")
print(f"Train augmented - NaN: {np.isnan(x_train_flat).any()}, Inf: {np.isinf(x_train_flat).any()}")
print(f"Train clean - NaN: {np.isnan(x_train_clean_flat).any()}, Inf: {np.isinf(x_train_clean_flat).any()}")
print(f"Train labels - NaN: {np.isnan(y_train_augmented).any()}")
print(f"Value ranges - Augmented: [{x_train_flat.min():.3f}, {x_train_flat.max():.3f}]")
print(f"Value ranges - Clean: [{x_train_clean_flat.min():.3f}, {x_train_clean_flat.max():.3f}]")

Loading augmented data...
✓ Train shape: (150000, 32, 32, 3)
✓ Test shape: (30000, 32, 32, 3)
Creating correct clean targets using original_index...
✓ Train flat shape: (150000, 3072)
✓ Train clean flat shape: (150000, 3072)
✓ Test flat shape: (30000, 3072)
✓ Test clean flat shape: (30000, 3072)

[Data Validation]
Train augmented - NaN: False, Inf: False
Train clean - NaN: False, Inf: False
Train labels - NaN: False
Value ranges - Augmented: [0.000, 1.000]
Value ranges - Clean: [0.000, 1.000]


## 8. 모델 임포트

In [4]:
# 모델 임포트 (models 폴더에서)
from models.bam_sequential import SequentialBAM

# 다른 모델들 (있다면)
try:
    from models.cae_sequential import SequentialCAE
    print("✓ Sequential CAE imported")
except:
    print("⚠ Sequential CAE not found (optional)")

try:
    from models.unet_sequential import SequentialUNet
    print("✓ Sequential U-Net imported")
except:
    print("⚠ Sequential U-Net not found (optional)")

print("\n✓ BAM model imported successfully")

✓ Sequential CAE imported
✓ Sequential U-Net imported

✓ BAM model imported successfully


## 9. 학습 시작 - Sequential BAM

### 9.1 Sequential BAM

In [5]:
print("\n" + "="*80)
print("Model 1/3: Sequential BAM")
print("="*80)

# 🔍 디버깅: 작은 배치로 먼저 테스트
print("\n[Pre-flight Check: Testing with small batch]")
test_batch_size = 256
x_test_batch = x_train_flat[:test_batch_size]
y_test_batch = x_train_clean_flat[:test_batch_size]

print(f"Test batch shape: {x_test_batch.shape}")
print(f"Test batch - NaN: {np.isnan(x_test_batch).any()}, Inf: {np.isinf(x_test_batch).any()}")
print(f"Test batch - Min: {x_test_batch.min():.6f}, Max: {x_test_batch.max():.6f}, Mean: {x_test_batch.mean():.6f}")
print(f"Target batch - NaN: {np.isnan(y_test_batch).any()}, Inf: {np.isinf(y_test_batch).any()}")
print(f"Target batch - Min: {y_test_batch.min():.6f}, Max: {y_test_batch.max():.6f}, Mean: {y_test_batch.mean():.6f}")

# 모델 생성
seq_bam = SequentialBAM(
    input_dim=3072,  # 32*32*3
    denoise_latent=256,
    cls_latent=128,
    num_classes=10
)

# 모델 컴파일
seq_bam.compile_models( 
    denoise_lr=INITIAL_LR,
    cls_lr=INITIAL_LR
)

print("\n[Stage 1: Denoising BAM]")
seq_bam.denoise_model.summary()

# 🔍 디버깅: 단일 배치로 forward pass 테스트
print("\n[Testing forward pass with single batch...]")
try:
    test_output = seq_bam.denoise_model.predict(x_test_batch[:32], verbose=0)
    print(f"✓ Forward pass successful")
    print(f"  Output shape: {test_output.shape}")
    print(f"  Output - NaN: {np.isnan(test_output).any()}, Inf: {np.isinf(test_output).any()}")
    print(f"  Output range: [{test_output.min():.6f}, {test_output.max():.6f}]")
except Exception as e:
    print(f"❌ Forward pass failed: {e}")

# Stage 1 학습
callbacks_stage1 = get_callbacks(
    'sequential_bam_stage1', 
    monitor='val_mse', 
    patience=30,
    initial_lr=INITIAL_LR,
    epochs=EPOCHS
)

history_stage1 = seq_bam.train_stage1(
    x_train_flat,
    x_train_clean_flat,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage1
)

# Stage 1 결과 저장
save_history(history_stage1, 'sequential_bam_stage1')
seq_bam.denoise_model.save('weights/sequential_bam_denoise.keras')
print("✓ Stage 1 model saved")

# ===== 메모리 정리 (OOM 방지) =====
import gc
gc.collect()
print("✓ Memory cleaned before Stage 2")
# ==================================

print("\n[Stage 2: Classification BAM]")
seq_bam.cls_model.summary()

# Stage 2 학습
callbacks_stage2 = get_callbacks(
    'sequential_bam_stage2', 
    monitor='val_accuracy', 
    patience=30,
    initial_lr=INITIAL_LR,
    epochs=EPOCHS
)

history_stage2 = seq_bam.train_stage2(
    x_train_flat,
    y_train_augmented,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage2
)

# Stage 2 결과 저장
save_history(history_stage2, 'sequential_bam_stage2')
seq_bam.cls_model.save('weights/sequential_bam_classification.keras')
print("✓ Stage 2 model saved")

# 평가  
print("\n[Evaluation]")
results = seq_bam.evaluate(
    x_test_flat, 
    x_test_clean_flat, 
    y_test_augmented, 
    batch_size=BATCH_SIZE
)
save_results(results, 'sequential_bam')

print("\nSequential BAM Results:")
print(f"  Restoration MSE: {results['restoration']['mse']:.6f}")
print(f"  Restoration PSNR: {results['restoration']['psnr']:.2f} dB")
print(f"  Classification Accuracy: {results['classification']['accuracy']:.4f}")

# 메모리 정리
del seq_bam
clear_memory()


Model 1/3: Sequential BAM

[Pre-flight Check: Testing with small batch]
Test batch shape: (256, 3072)
Test batch - NaN: False, Inf: False
Test batch - Min: 0.000000, Max: 1.000000, Mean: 0.474153
Target batch - NaN: False, Inf: False
Target batch - Min: 0.000000, Max: 1.000000, Mean: 0.473305


2025-11-03 12:27:55.388917: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-03 12:27:55.495777: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-03 12:27:55.495824: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-03 12:27:55.498115: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-03 12:27:55.498152: I external/local_xla/xla/stream_executor

✓ Models compiled successfully (with gradient clipping)

[Stage 1: Denoising BAM]
Model: "denoise_bam"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 noisy_input (InputLayer)    [(None, 3072)]            0         
                                                                 
 mf (MF)                     ((None, 3072),            8935424   
                              (None, 256))                       
                                                                 
Total params: 8935424 (34.09 MB)
Trainable params: 8925696 (34.05 MB)
Non-trainable params: 9728 (38.00 KB)
_________________________________________________________________

[Testing forward pass with single batch...]
✓ Forward pass successful
  Output shape: (32, 3072)
  Output - NaN: False, Inf: False
  Output range: [0.335085, 0.650273]

Stage 1: Training Restoration MF
Architecture: MF (Multifactor) Autoencoder
Input dim: 3072

2025-11-03 12:28:03.791471: I external/local_xla/xla/service/service.cc:168] XLA service 0x7faacfbcbbe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-03 12:28:03.791512: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-11-03 12:28:03.798058: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-03 12:28:03.822388: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8905
I0000 00:00:1762140483.892195  279131 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


936/938 [============================>.] - ETA: 0s - loss: 0.2115 - mse: 0.0669 - mae: 0.2115
Epoch 1: val_mse improved from inf to 0.06309, saving model to weights/sequential_bam_stage1_best.keras
938/938 [==============================] - 19s 16ms/step - loss: 0.2115 - mse: 0.0669 - mae: 0.2115 - val_loss: 0.2063 - val_mse: 0.0631 - val_mae: 0.2063 - lr: 0.0010
Epoch 2/200
938/938 [==============================] - ETA: 0s - loss: 0.2068 - mse: 0.0632 - mae: 0.2068
Epoch 2: val_mse improved from 0.06309 to 0.06203, saving model to weights/sequential_bam_stage1_best.keras
938/938 [==============================] - 14s 15ms/step - loss: 0.2068 - mse: 0.0632 - mae: 0.2068 - val_loss: 0.2055 - val_mse: 0.0620 - val_mae: 0.2055 - lr: 9.8855e-04
Epoch 3/200
937/938 [============================>.] - ETA: 0s - loss: 0.2058 - mse: 0.0624 - mae: 0.2058
Epoch 3: val_mse did not improve from 0.06203
938/938 [==============================] - 13s 14ms/step - loss: 0.2058 - mse: 0.0624 - mae: 0.2

### ✅ NaN 문제 해결 요약

**문제 원인:**
1. ❌ Clean target 매칭 오류 (셔플된 데이터와 순서 불일치)
2. ❌ BatchNormalization 불안정 (validation 시)
3. ❌ Cubic activation의 수치 불안정성
4. ❌ Gradient explosion

**해결 방법:**
1. ✅ `original_index`로 정확한 clean target 매칭
2. ✅ BatchNormalization 파라미터 안정화 (momentum=0.99, epsilon=1e-3)
3. ✅ Cubic activation clipping (-10 ~ 10)
4. ✅ 입력/출력/잠재 표현 clipping
5. ✅ Gradient clipping (clipnorm=1.0, clipvalue=0.5)
6. ✅ 가중치 초기화 개선 (glorot_uniform)

**다음 단계:**
- Cell 6 실행 → 데이터 검증
- Cell 10 실행 → 디버깅 + 학습 시작


### 🔧 GPU 메모리 부족(OOM) 문제 해결

**문제 상황:**
```
ResourceExhaustedError: OOM when allocating tensor with shape[150000,32,32,3]
```

**원인:**
- CAE/U-Net의 `train_stage2`에서 150,000개 이미지를 한 번에 메모리에 로드
- `x_restored = model.predict(x_noisy)` → 전체 데이터를 한 번에 처리

**해결 방법:**
1. ✅ **배치 단위 처리**: chunk_size = batch_size × 4로 나눠서 처리
2. ✅ **메모리 정리**: `gc.collect()` 추가
3. ✅ **진행 상황 표시**: 10 chunk마다 로그 출력

**수정된 파일:**
- `models/cae_sequential.py` - train_stage2 메서드
- `models/unet_sequential.py` - train_stage2 메서드

**메모리 절약:**
- 기존: 150,000 × 32 × 32 × 3 × 4 bytes ≈ **5.5 GB** (한 번에)
- 수정: 512 × 32 × 32 × 3 × 4 bytes ≈ **19 MB** (청크당)

이제 Jupyter 커널을 재시작하고 다시 실행하세요!


### 6.2 Sequential CAE

In [ ]:
print("\n" + "="*80)
print("Model 2/6: Sequential CAE")
print("="*80)

# 모델 생성
seq_cae = SequentialCAE(
    input_shape=(32, 32, 3),
    num_classes=10
)

# 모델 컴파일
seq_cae.compile_models(
    restore_lr=INITIAL_LR,
    cls_lr=INITIAL_LR,
    restore_loss='mae'
)

print("\n[Stage 1: Restoration CAE]")
seq_cae.restore_model.summary()

# ✅ 수정: original_index로 올바르게 매칭된 clean target 사용
print("\n⚠️ Using correctly matched clean targets (already created in Cell 6)")
print(f"✓ x_train_clean_matched shape: {x_train_clean_matched.shape}")
print(f"✓ x_test_clean_matched shape: {x_test_clean_matched.shape}")

# Stage 1 학습
callbacks_stage1 = get_callbacks('sequential_cae_stage1', monitor='val_mae', patience=30, initial_lr=INITIAL_LR, epochs=EPOCHS)
history_stage1 = seq_cae.train_stage1(
    x_train_augmented,
    x_train_clean_matched,  # ✅ 수정: 올바른 매칭 사용
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage1
)

# Stage 1 결과 저장
save_history(history_stage1, 'sequential_cae_stage1')
seq_cae.restore_model.save('weights/sequential_cae_restore.keras')
print("✓ Stage 1 model saved")

# ===== 메모리 정리 (OOM 방지) =====
import gc
gc.collect()
print("✓ Memory cleaned before Stage 2")
# ==================================

print("\n[Stage 2: Classification CAE]")
seq_cae.cls_model.summary()

# Stage 2 학습
callbacks_stage2 = get_callbacks('sequential_cae_stage2', monitor='val_accuracy', patience=30, initial_lr=INITIAL_LR, epochs=EPOCHS)
history_stage2 = seq_cae.train_stage2(
    x_train_augmented,
    y_train_augmented,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage2
)

# Stage 2 결과 저장
save_history(history_stage2, 'sequential_cae_stage2')
seq_cae.cls_model.save('weights/sequential_cae_classification.keras')
print("✓ Stage 2 model saved")

# 평가
print("\n[Evaluation]")
results = seq_cae.evaluate(
    x_test_augmented,
    x_test_clean_matched,  # ✅ 수정: 올바른 매칭 사용
    y_test_augmented,
    batch_size=BATCH_SIZE
)
save_results(results, 'sequential_cae')

print("\nSequential CAE Results:")
print(f"  Restoration MSE: {results['restoration']['mse']:.6f}")
print(f"  Restoration PSNR: {results['restoration']['psnr']:.2f} dB")
print(f"  Classification Accuracy: {results['classification']['accuracy']:.4f}")

# 메모리 정리
del seq_cae
clear_memory()


Model 2/6: Sequential CAE

[Stage 1: Restoration CAE]
Model: "CAE_restoration"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928  

2025-11-03 12:54:51.455079: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-11-03 12:54:51.492886: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


938/938 [==============================] - ETA: 0s - loss: 0.2068 - mse: 0.0626 - mae: 0.2068
Epoch 1: val_mae improved from inf to 0.20597, saving model to weights/sequential_cae_stage1_best.keras
938/938 [==============================] - 30s 25ms/step - loss: 0.2068 - mse: 0.0626 - mae: 0.2068 - val_loss: 0.2060 - val_mse: 0.0619 - val_mae: 0.2060 - lr: 0.0010
Epoch 2/200
937/938 [============================>.] - ETA: 0s - loss: 0.2066 - mse: 0.0625 - mae: 0.2066
Epoch 2: val_mae improved from 0.20597 to 0.20594, saving model to weights/sequential_cae_stage1_best.keras
938/938 [==============================] - 19s 21ms/step - loss: 0.2066 - mse: 0.0625 - mae: 0.2066 - val_loss: 0.2059 - val_mse: 0.0620 - val_mae: 0.2059 - lr: 9.8855e-04
Epoch 3/200
937/938 [============================>.] - ETA: 0s - loss: 0.2066 - mse: 0.0625 - mae: 0.2066
Epoch 3: val_mae did not improve from 0.20594
938/938 [==============================] - 19s 20ms/step - loss: 0.2066 - mse: 0.0625 - mae: 0.2

KeyboardInterrupt: 

### 6.3 Sequential U-Net

In [ ]:
print("\n" + "="*80)
print("Model 3/6: Sequential U-Net")
print("="*80)

# 모델 생성
seq_unet = SequentialUNet(
    input_shape=(32, 32, 3),
    num_classes=10,
    restore_dropout=0.0,
    cls_dropout=0.1
)

# 모델 컴파일
seq_unet.compile_models(
    restore_lr=INITIAL_LR,
    cls_lr=INITIAL_LR,
    restore_loss='mae'
)

print("\n[Stage 1: Restoration U-Net]")
seq_unet.restore_model.summary()

# ✅ 수정: 이미 Cell 6에서 생성된 매칭된 데이터 재사용
print("\n⚠️ Using correctly matched clean targets (from Cell 6)")

# Stage 1 학습
callbacks_stage1 = get_callbacks('sequential_unet_stage1', monitor='val_mae', patience=30, initial_lr=INITIAL_LR, epochs=EPOCHS)
history_stage1 = seq_unet.train_stage1(
    x_train_augmented,
    x_train_clean_matched,  # ✅ 수정: 올바른 매칭 사용
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage1
)

# Stage 1 결과 저장
save_history(history_stage1, 'sequential_unet_stage1')
seq_unet.restore_model.save('weights/sequential_unet_restore.keras')
print("✓ Stage 1 model saved")

# ===== 메모리 정리 (OOM 방지) =====
import gc
gc.collect()
print("✓ Memory cleaned before Stage 2")
# ==================================

print("\n[Stage 2: Classification U-Net]")
seq_unet.cls_model.summary()

# Stage 2 학습
callbacks_stage2 = get_callbacks('sequential_unet_stage2', monitor='val_accuracy', patience=30, initial_lr=INITIAL_LR, epochs=EPOCHS)
history_stage2 = seq_unet.train_stage2(
    x_train_augmented,
    y_train_augmented,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks_stage2
)

# Stage 2 결과 저장
save_history(history_stage2, 'sequential_unet_stage2')
seq_unet.cls_model.save('weights/sequential_unet_classification.keras')
print("✓ Stage 2 model saved")

# 평가
print("\n[Evaluation]")
results = seq_unet.evaluate(
    x_test_augmented,
    x_test_clean_matched,  # ✅ 수정: 올바른 매칭 사용
    y_test_augmented,
    batch_size=BATCH_SIZE
)
save_results(results, 'sequential_unet')

print("\nSequential U-Net Results:")
print(f"  Restoration MSE: {results['restoration']['mse']:.6f}")
print(f"  Restoration PSNR: {results['restoration']['psnr']:.2f} dB")
print(f"  Classification Accuracy: {results['classification']['accuracy']:.4f}")

# 메모리 정리
del seq_unet
clear_memory()

## 7. Sequential 모델 학습 완료

3개의 Sequential 모델 학습이 완료되었습니다.

다음 Part 3에서 MTL 모델들을 학습하겠습니다.